In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import xgboost as xgb

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score,mean_squared_error
from math import sqrt
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge

def apply_log(train,test):
    train["Upvotes"] = np.log1p(train["Upvotes"])
    train["Reputation"] = np.log1p(train["Reputation"])
    train["Views"] = np.log1p(train["Views"])
    train["Answers"] = np.log1p(train["Answers"])

    test["Reputation"] = np.log1p(test["Reputation"])
    test["Views"] = np.log1p(test["Views"])
    test["Answers"] = np.log1p(test["Answers"])
    return train,test

def get_label_id(train,test):
    #Drop the irrelevant features from the test set
    ID_train = train['ID']
    train_labels = np.array(train['Upvotes'])
    train_features = train.drop(['Upvotes','ID','Username','Tag'], axis=1)
    train_features = train_features.fillna(0)
    #Drop the irrelevant features from the test set
    ID_test = test['ID']
    test_features = test.drop(['ID','Username','Tag'], axis=1)
    test_features = test_features.fillna(0)
    
    return ID_train,train_features,train_labels,ID_test,test_features
    

def train_mod(train_features,test_features,train_labels):
    st_train = train_features.values
    st_test = test_features.values
    Y = train_labels
#     clf = lgb.LGBMRegressor()
#     clf = xgb.XGBRegressor()
#     clf = LinearRegression()
    clf = BayesianRidge()
    # clf = lgb.LGBMRegressor(max_depth=6,learning_rate=0.0716,n_estimators=128,num_leaves=24,reg_alpha=1.7250,reg_lambda=0.0888,subsample=0.6361,colsample_bytree=0.9365)
#     clf = xgb.XGBRegressor(gamma = 0.76,learning_rate = 0.0100,max_depth = 5,min_child_weight = 2,n_estimators = 107,subsample = 0.60,colsample_bytree = 0.9900)
#     clf = xgb.XGBRegressor(gamma = 0.9257,learning_rate = 0.2797,max_depth = 5,min_child_weight = 9,n_estimators = 305,subsample = 0.6239,colsample_bytree = 0.7443)
#     clf = xgb.XGBRegressor(gamma = 0.9019,learning_rate = 0.2530,max_depth = 4,min_child_weight = 8,n_estimators = 300,subsample = 0.6409,colsample_bytree = 0.7380)
    fold = 5
    cv = KFold(n_splits=fold, shuffle=True, random_state=42)
    X_preds = np.zeros(st_train.shape[0])
    preds = np.zeros(st_test.shape[0])
    for i, (tr, ts) in enumerate(cv.split(st_train)):
        print(ts.shape)
        mod = clf.fit(st_train[tr], Y[tr])
        X_preds[ts] = mod.predict(st_train[ts])
        preds += mod.predict(st_test)
        print("fold {}, RMSE : {:.3f}".format(i, sqrt(mean_squared_error(Y[ts], X_preds[ts]))))
    score = sqrt(mean_squared_error(Y, X_preds))
    print(score)
    preds1 = preds/fold
    preds1 = np.abs(np.expm1(preds1))
    X_preds = np.abs(np.expm1(X_preds))
    return X_preds,preds1

def save_off(train_ID,X_preds,test_id,preds1):
    tr_sub = pd.DataFrame({'ID': train_ID, 'Upvotes': X_preds})
    tr_sub=tr_sub.reindex(columns=["ID","Upvotes"])
    tr_sub.to_csv('train_oof.csv', index=False)

    sub = pd.DataFrame({'ID': test_id, 'Upvotes': preds1})
    sub=sub.reindex(columns=["ID","Upvotes"])
    sub.to_csv('submission.csv', index=False)

train = pd.read_csv('../input/train_NIR5Yl1.csv')
test = pd.read_csv('../input/test_8i3B3FC.csv')

test_id=[]
train_id=[]
test_pred=[]
train_pred=[]

print(train.shape)
print(test.shape)
print(train.columns)
print(train.Tag.unique())

#train1 = train1.drop(train1[(train1['Views']>3100000) | (train1['Reputation'] > 900000) | (train1['Upvotes'] > 210000) | (train1['Answers'] > 65)].index)
#array(['a', 'c', 'r', 'j', 'p', 's', 'h', 'o', 'i', 'x']

##a##
train1 = train[train.Tag == 'a']
test1 = test[test.Tag == 'a']
print("Before train1 drop: ",train1.shape)
print("Before test1 drop",test1.shape)
train1 = train1.drop(train1[(train1['Views']>2000000) | (train1['Upvotes'] > 100000) | (train1['Answers'] > 50)].index)
print("After train1 drop: ",train1.shape)

train1,test1 = apply_log(train1,test1)
ID_train,train_features,train_labels,ID_test,test_features = get_label_id(train1,test1)
X_preds,preds1 = train_mod(train_features,test_features,train_labels)

test_id.extend(ID_test)
train_id.extend(ID_train)
test_pred.extend(preds1)
train_pred.extend(X_preds)

##c##
train1 = train[train.Tag == 'c']
test1 = test[test.Tag == 'c']
train1 = train1.drop(train1[(train1['Views']>1700000) | (train1['Upvotes'] > 150000) | (train1['Answers'] > 50)].index)

train1,test1 = apply_log(train1,test1)
ID_train,train_features,train_labels,ID_test,test_features = get_label_id(train1,test1)
X_preds,preds1 = train_mod(train_features,test_features,train_labels)

test_id.extend(list(ID_test))
train_id.extend(list(ID_train))
test_pred.extend(list(preds1))
train_pred.extend(list(X_preds))

##r##
train1 = train[train.Tag == 'r']
test1 = test[test.Tag == 'r']
train1 = train1.drop(train1[(train1['Views']>600000) | (train1['Upvotes'] > 80000) | (train1['Answers'] > 20)].index)

train1,test1 = apply_log(train1,test1)
ID_train,train_features,train_labels,ID_test,test_features = get_label_id(train1,test1)
X_preds,preds1 = train_mod(train_features,test_features,train_labels)

test_id.extend(list(ID_test))
train_id.extend(list(ID_train))
test_pred.extend(list(preds1))
train_pred.extend(list(X_preds))

##j##
train1 = train[train.Tag == 'j']
test1 = test[test.Tag == 'j']
train1 = train1.drop(train1[(train1['Views']>3000000) | (train1['Upvotes'] > 300000) | (train1['Answers'] > 60) | (train1['Reputation'] > 700000)].index)

train1,test1 = apply_log(train1,test1)
ID_train,train_features,train_labels,ID_test,test_features = get_label_id(train1,test1)
X_preds,preds1 = train_mod(train_features,test_features,train_labels)

test_id.extend(list(ID_test))
train_id.extend(list(ID_train))
test_pred.extend(list(preds1))
train_pred.extend(list(X_preds))

##p##
train1 = train[train.Tag == 'p']
test1 = test[test.Tag == 'p']
train1 = train1.drop(train1[(train1['Views']>2300000) | (train1['Upvotes'] > 160000) | (train1['Answers'] > 39) | (train1['Reputation'] > 600000)].index)

train1,test1 = apply_log(train1,test1)
ID_train,train_features,train_labels,ID_test,test_features = get_label_id(train1,test1)
X_preds,preds1 = train_mod(train_features,test_features,train_labels)

test_id.extend(list(ID_test))
train_id.extend(list(ID_train))
test_pred.extend(list(preds1))
train_pred.extend(list(X_preds))

##s##
train1 = train[train.Tag == 's']
test1 = test[test.Tag == 's']
train1 = train1.drop(train1[(train1['Views']>2500000) | (train1['Upvotes'] > 160000) | (train1['Reputation'] > 630000)].index)

train1,test1 = apply_log(train1,test1)
ID_train,train_features,train_labels,ID_test,test_features = get_label_id(train1,test1)
X_preds,preds1 = train_mod(train_features,test_features,train_labels)

test_id.extend(list(ID_test))
train_id.extend(list(ID_train))
test_pred.extend(list(preds1))
train_pred.extend(list(X_preds))

##h##
train1 = train[train.Tag == 'h']
test1 = test[test.Tag == 'h']
train1 = train1.drop(train1[(train1['Views']>2200000) | (train1['Upvotes'] > 150000) | (train1['Answers'] > 42) | (train1['Reputation'] > 500000)].index)

train1,test1 = apply_log(train1,test1)
ID_train,train_features,train_labels,ID_test,test_features = get_label_id(train1,test1)
X_preds,preds1 = train_mod(train_features,test_features,train_labels)

test_id.extend(list(ID_test))
train_id.extend(list(ID_train))
test_pred.extend(list(preds1))
train_pred.extend(list(X_preds))

##o##
train1 = train[train.Tag == 'o']
test1 = test[test.Tag == 'o']
train1 = train1.drop(train1[(train1['Views']>450000) | (train1['Upvotes'] > 20000) | (train1['Reputation'] > 280000)].index)

train1,test1 = apply_log(train1,test1)
ID_train,train_features,train_labels,ID_test,test_features = get_label_id(train1,test1)
X_preds,preds1 = train_mod(train_features,test_features,train_labels)

test_id.extend(list(ID_test))
train_id.extend(list(ID_train))
test_pred.extend(list(preds1))
train_pred.extend(list(X_preds))

##i##
train1 = train[train.Tag == 'i']
test1 = test[test.Tag == 'i']
train1 = train1.drop(train1[(train1['Views']>550000) | (train1['Upvotes'] > 22000) | (train1['Reputation'] > 400000)].index)

train1,test1 = apply_log(train1,test1)
ID_train,train_features,train_labels,ID_test,test_features = get_label_id(train1,test1)
X_preds,preds1 = train_mod(train_features,test_features,train_labels)

test_id.extend(list(ID_test))
train_id.extend(list(ID_train))
test_pred.extend(list(preds1))
train_pred.extend(list(X_preds))

##x##
train1 = train[train.Tag == 'x']
test1 = test[test.Tag == 'x']
train1 = train1.drop(train1[(train1['Views']>600000) | (train1['Upvotes'] > 20000) | (train1['Answers'] > 28) | (train1['Reputation'] > 400000)].index)

train1,test1 = apply_log(train1,test1)
ID_train,train_features,train_labels,ID_test,test_features = get_label_id(train1,test1)
X_preds,preds1 = train_mod(train_features,test_features,train_labels)

test_id.extend(list(ID_test))
train_id.extend(list(ID_train))
test_pred.extend(list(preds1))
train_pred.extend(list(X_preds))

##end of all tags##

print("test_id length: " ,len(test_id))
print("test_pred length: " ,len(test_pred))
print("train_id length: " ,len(train_id))
print("train_pred length: " ,len(train_pred))

save_off(train_id,train_pred,test_id,test_pred)
print('Complete')


#xgboost bayesian optimization
from sklearn.cross_validation import cross_val_score
from bayes_opt import BayesianOptimization

train2 = train_features.values
Y = train_labels

def xgboostcv(max_depth,learning_rate,n_estimators,gamma,min_child_weight,subsample,colsample_bytree):
    return cross_val_score(xgb.XGBRegressor(max_depth=int(max_depth),learning_rate=learning_rate,n_estimators=int(n_estimators),
                                             silent=True,nthread=-1,gamma=gamma,min_child_weight=min_child_weight,
                                           subsample=subsample,colsample_bytree=colsample_bytree),
                           train2,Y,"neg_mean_squared_error",cv=5).mean()

xgboostBO = BayesianOptimization(xgboostcv,{'max_depth': (5, 10),'learning_rate': (0.01, 0.3),'n_estimators': (50, 1200),
                                  'gamma': (0.01,1.0),'min_child_weight': (2, 10),
                                            'subsample': (0.6, 0.8),'colsample_bytree' :(0.5, 0.99)})

xgboostBO.maximize()
print('-'*53)
print('Final Results')
print('XGBOOST: %f' % xgboostBO.res['max']['max_val'])

#lightgbm bayesian optimization
from sklearn.cross_validation import cross_val_score
from bayes_opt import BayesianOptimization

train2 = train_features.values
Y = train_labels

def xgboostcv(max_depth,learning_rate,n_estimators,num_leaves,reg_alpha,reg_lambda,subsample,colsample_bytree):
    return cross_val_score(lgb.LGBMRegressor(max_depth=int(max_depth),learning_rate=learning_rate,n_estimators=int(n_estimators),
                                             silent=True,nthread=-1,num_leaves=int(num_leaves),reg_alpha=reg_alpha,
                                           reg_lambda=reg_lambda,subsample=subsample,colsample_bytree=colsample_bytree),
                           train2,Y,"r2",cv=5).mean()

xgboostBO = BayesianOptimization(xgboostcv,{'max_depth': (2, 10),'learning_rate': (0.001, 0.1),'n_estimators': (10, 900),
                                  'num_leaves': (3,30),'reg_alpha': (1, 5),'reg_lambda': (0, 0.1),
                                            'subsample': (0.4, 0.8),'colsample_bytree' :(0.4, 0.99)})

xgboostBO.maximize()
print('-'*53)
print('Final Results')
print('XGBOOST: %f' % xgboostBO.res['max']['max_val'])